In [1]:
import pandas as pd
import requests
import secret

In [2]:
API_KEY = ''
#https://learndataanalysis.org/google-geocoding-api-tutorial-for-python/
def getLatLng(address):
    params = {
        'key': secret.REACT_APP_GOOGLE_MAPS_API_KEY,
        'address': address.replace(' ', '+')
    }

    api = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(api, params=params)
    data = response.json()
    if data['status'] == 'OK':
        print(f"Processed : {address}")
        result = data['results'][0]
        location = result['geometry']['location']
        return (location['lat'], location['lng'])
    else:
        #if we cannot find then do this
        return ("NaN", "NaN")

In [ ]:
df = pd.read_csv ("hospital_list.csv", index_col = 0)

In [ ]:
#we replace the NaN values with empty string for columsn Region, Province and City
# this is to get the actual place name
df['CITY'].fillna('', inplace = True)
df['REGION'].fillna('', inplace = True)
df['PROVINCE'].fillna('', inplace = True)

In [ ]:
df["Mapping_Address"] = df["Actual Hospital Name"] + ", " + df["REGION"] + ", " + df["PROVINCE"] +  ", " + df["CITY"]

In [ ]:
def cleanMapAddress (x):
    x = str(x)
    x = x.strip()
    if x[-1] == ",":
        return x[:-1]
    else:
        return x

In [ ]:
def replaceEmptyString (x):
    if x == '':
        return "N/A"

In [ ]:
def cleanEmail (email):
    result = ""
    for i in email:
        if i == " ":
            continue
        result += i
    return result

In [ ]:
df["Mapping_Address"] = df["Mapping_Address"].map(lambda x : cleanMapAddress (x))

In [ ]:
#fill back with "NaN"
df['CITY'].map (lambda x : replaceEmptyString (x))
df['REGION'].map (lambda x : replaceEmptyString (x))
df['PROVINCE'].map (lambda x : replaceEmptyString (x))
df.fillna ("NaN", inplace = True)
df.rename (columns={"OWNERSHIP CLASS.": "OWNERSHIP_CLASS", 
                    "MEDICAL DIRECTOR/ CHIEF OF HOSPITAL": "MEDICAL_DIRECTOR",
                   "SERVCE CAPABILITY" : "SERVCE_CAPABILITY",
                   "CONTACT NOS." : "CONTACT",
                   "E-MAIL ADDRESS": "EMAIL",
                   "FAX NO.": "FAX",
                   "MOBILE NO.": "MOBILE"}, inplace = True)

df['EMAIL'] = df['EMAIL'].map (lambda x : cleanEmail (x))

In [ ]:
df.head()

In [ ]:
df['lat'] = df["Mapping_Address"].map (lambda x : getLatLng(x)[0])
df['lng'] = df["Mapping_Address"].map (lambda x : getLatLng(x)[1])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#remove duplicated rows
df = df.drop_duplicates(keep='first')

In [ ]:
df.shape

In [ ]:
no_coord_df = df[(df['lat'] == "NaN") | (df['lng'] == "NaN")]
no_coord_df.shape

In [ ]:
coord_df = df[(df['lat'] != "NaN") & (df['lng'] != "NaN")]
coord_df.shape

In [ ]:
coord_df.to_csv ("mapping_data_hospital.csv", index = False)

In [ ]:
no_coord_df.to_csv ("no_mapping_data_hospital.csv", index = False)